In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences


IMAGE_SIZE = (416, 416)  # 모델 입력 크기 (예: 224x224)

# 이미지와 클래스 값만 로드
def load_data(image_folder, label_folder):
    images = []
    labels = []

    for image_name in os.listdir(image_folder):
        if image_name.endswith(".jpg"):
            # 이미지 경로
            image_path = os.path.join(image_folder, image_name)

            # 라벨 경로
            label_name = image_name.replace(".jpg", ".txt")
            label_path = os.path.join(label_folder, label_name)

            # 이미지 로드
            image = load_img(image_path, target_size=IMAGE_SIZE)
            image = img_to_array(image) / 255.0  # 정규화
            images.append(image)

            # 라벨 로드 (클래스 값만 추출)
            classes = []
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f.readlines():
                        class_id = int(line.split()[0])  # 첫 번째 값이 클래스 ID
                        classes.append(class_id)
            labels.append(classes)

    labels = pad_sequences(labels, padding='post', maxlen=3)
    
    # Tensor 변환
    tensor = tf.convert_to_tensor(labels, dtype=tf.float32)
    return np.array(images), tensor

: 

In [3]:

# GPU 사용 여부 확인
print("Is GPU available?:", tf.config.list_physical_devices('GPU'))

Is GPU available?: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
from PIL import Iamge

ImportError: cannot import name 'Iamge' from 'PIL' (/home/user/anaconda3/envs/EAI/lib/python3.10/site-packages/PIL/__init__.py)

In [7]:
X_train, y_train = load_data('train/images', 'train/labels')

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
X_test, y_test = load_data('test/images', 'test/labels')

In [ ]:
# 간단한 다중 라벨 분류 모델 정의
def create_multi_label_model(input_shape, num_labels):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_labels, activation='sigmoid')  # 다중 라벨
    ])
    return model


In [ ]:
# 다중 라벨 데이터셋 준비
def create_multi_label_dataset(images, multi_labels, batch_size=32):
    """
    다중 라벨 데이터셋 생성 함수.
    
    Args:
        images (np.array): 입력 이미지 배열.
        multi_labels (np.array): 다중 라벨 배열 (One-hot 형식).
        batch_size (int): 배치 크기.

    Returns:
        dataset (tf.data.Dataset): TensorFlow 데이터셋.
    """
    dataset = tf.data.Dataset.from_tensor_slices((images, multi_labels))
    dataset = dataset.shuffle(len(images)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
def train_multi_label_model(model, dataset, val_dataset=None, epochs=10, learning_rate=0.001):
    """
    다중 라벨 분류를 위한 학습 함수.

    Args:
        model (tf.keras.Model): 학습할 모델.
        dataset (tf.data.Dataset): 훈련 데이터셋. (입력 이미지, 다중 라벨 형식)
        val_dataset (tf.data.Dataset, optional): 검증 데이터셋. Default is None.
        epochs (int): 학습 반복 횟수. Default is 10.
        learning_rate (float): 학습률. Default is 0.001.

    Returns:
        history (History): 학습 결과.
    """
    # 옵티마이저와 손실 함수 정의
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss_fn = tf.keras.losses.BinaryCrossentropy()  # 다중 라벨 분류를 위한 손실 함수

    # 모델 컴파일
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

    # 모델 학습
    history = model.fit(
        dataset,
        validation_data=val_dataset,
        epochs=epochs
    )

    return history


In [ ]:
# 데이터셋 생성
train_dataset = create_multi_label_dataset(X_train, y_train, batch_size=32)
val_dataset = create_multi_label_dataset(X_test, y_test, batch_size=32)  # 예: 일부 검증 데이터

# 모델 생성
input_shape = (416, 416, 3)  # 이미지 크기
num_labels = y_train.shape[1]  # 라벨 개수 (다중 라벨의 차원)
model = create_multi_label_model(input_shape, num_labels)

# 모델 학습
history = train_multi_label_model(model, train_dataset, val_dataset, epochs=10, learning_rate=0.001)

# 학습 결과 확인
import matplotlib.pyplot as plt

# 학습 곡선 시각화
plt.plot(history.history['loss'], label='Train Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()

plt.plot(history.history['accuracy'], label='Train Accuracy')
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()


2024-11-23 17:55:40.486857: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6977617920 exceeds 10% of free system memory.
2024-11-23 17:55:46.067044: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6977617920 exceeds 10% of free system memory.
2024-11-23 17:55:50.086052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [3360,3]
	 [[{{node Placeholder/_1}}]]
2024-11-23 17:55:50.086398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [3360,416,416,3]
	 [[{{node Placeholder/_0}}]]


Epoch 1/10
 32/105 [========>.....................] - ETA: 2:21 - loss: -3679.2207 - accuracy: 0.9688